In [83]:
import requests 
import numpy as np  
from bs4 import BeautifulSoup
import pandas as pd

In [90]:
exchanges=['shtnyse', 'shtnasdaq','shtamex']
datas=['0_9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [130]:
def parse_thread(exchange, company_initial, container, header_container):
 
    url = "http://www.wsj.com/mdc/public/page/2_3062-{exchange}_{company_initial}-listing.html".format(
        exchange = exchange,
        company_initial = company_initial)
  
    res = requests.get(url)
    bs4_obj = BeautifulSoup(res.text.replace("\n",""), "html.parser")
    trs = bs4_obj.select("table tr")
    array_of_rows=  []
    for tr in trs:
        tds = tr.select("td")
        tds_data = [td.text for td in tds]
        array_of_rows.append(tds_data)
    array_of_rows = array_of_rows[2:]
    container += array_of_rows[1:-5]
    header_container[0] = array_of_rows[0]
   

In [185]:
import threading 
def run_with_thread():
    threads = []
    combined_data = []
    header_container = [None]
    import time
    for exchange in exchanges:
        print(exchange)
        for company_initial in datas:
            t = threading.Thread(target = parse_thread,args =(exchange, company_initial, combined_data, header_container,))
            t.start()
            threads.append(t)
            time.sleep(0.05)

    for t in threads:
        t.join()
    

    df = pd.DataFrame(combined_data, columns = header_container[0])
    return df
        

In [173]:
def parse(exchange, company_initial):
 
    url = "http://www.wsj.com/mdc/public/page/2_3062-{}_{}-listing.html".format(
        exchange,
        company_initial)
 
    res = requests.get(url)
    bs4_obj = BeautifulSoup(res.text.replace("\n",""), "html.parser")
    trs = bs4_obj.select("table tr")
    array_of_rows=  []
    for tr in trs:
        tds = tr.select("td")
        tds_data = [td.text for td in tds]
        if (len(tds_data)!=9):
            pass
        else:
           
            array_of_rows.append(tds_data)
            
            
    headers = array_of_rows[0] if len(array_of_rows)>=1 else []
    needed_data = array_of_rows[1:] if len(array_of_rows)>=2 else []
    
    return needed_data, headers
  
   

In [181]:
def run_without_thread():
    header = None
    combined_data = []
    for exchange in exchanges:
        print(exchange)
        for company_initial in datas:
            needed_data, headers = parse(exchange, company_initial)
            combined_data += needed_data
            header = headers
    df = pd.DataFrame(combined_data, columns = header)
    return df
    
        

In [186]:
import datetime
start = datetime.datetime.now()
df = run_with_thread()
finish = datetime.datetime.now()
print("run with thread spends = {}".format(finish-start))


shtnyse
shtnasdaq
shtamex
run with thread spends = 0:00:14.466025


In [182]:
start = datetime.datetime.now()
df = run_without_thread()
finish = datetime.datetime.now()
print("run without thread spends = {}".format(finish-start))

shtnyse
shtnasdaq
shtamex
run without thread spends = 0:02:39.815195


In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8178 entries, 0 to 8177
Data columns (total 9 columns):
Company            8178 non-null object
Symbol             8178 non-null object
3/15/19            8178 non-null object
2/28/19            8178 non-null object
Chg                8178 non-null object
% Chg              8178 non-null object
% Float            8178 non-null object
Days tocover       8178 non-null object
Avg dailyvolume    8178 non-null object
dtypes: object(9)
memory usage: 575.1+ KB
